In [1]:
import numpy as np

In [2]:
# Returns the index of the entering variable, the entering variable and it's reduced cost,
# or returns 0 if no entering variable exists
def fullfindEV(n, c, A, varstatus, pi_t, phase1):
    nonbasicvars = [index for index, value in enumerate(varstatus[0]) if value == -1]

    cn = c[nonbasicvars]
    cn_t = cn.T
    N = A[:,nonbasicvars]

    reduced_costs = cn_t - np.dot(pi_t, N)
    s = np.argmin(reduced_costs)
    minrc = reduced_costs[0][s]
    ev = nonbasicvars[s]

    return [s, ev, minrc]

In [3]:
#Returns the position in the basis of the leaving variable, leaving variable
#or returns 0 if no leaving variable exists
def fullfindLV(m, xB, BinvAs, basicvars, phase1):
    # print(xB, BinvAs)
    ratios = xB / (BinvAs + 1e-15)

    for i in range(ratios.shape[0]):
        if ratios[i] < 0:
            ratios[i] = 1e6

    r = np.argmin(ratios)
    minratio = ratios[r][0]
    lv = basicvars[0][r]

    return [r, lv, minratio]

In [4]:
#Updates the basis representation.
def fullupdate(m, c, s, r, BinvAs, varstatus, basicvars, cB, Binv, xB, phase1=True):
    gjp = np.hstack((xB, Binv, BinvAs))
    gjp[r] /= (gjp[r][-1] + 1e-15)

    for i in range(m):
        if i == r:
            continue
        gjp[i] -= gjp[r] * gjp[i][-1]

    Binv = gjp[:, 1:-1]

    nonbasicvars = [index for index, value in enumerate(varstatus[0]) if value == -1]

    ev = nonbasicvars[s]
    lv = basicvars[0][r]

    basicvars[0][r] = ev

    if phase1:
        if lv < n:
            varstatus[0][lv] = -1
        varstatus[0][ev] = r
    else:
        varstatus[0][lv] = -1
        varstatus[0][ev] = r

    xB = np.dot(Binv, b)
    cB = c[basicvars[0]]

    return [varstatus, basicvars, cB, Binv, xB, gjp[:, -1:]]

In [5]:
#Solves a linear program using Gauss-Jordon updates
# Assumes standard computational form
# Performs a Phase I procedure starting from an artificial basis
def fullrsm(m,n,c,A,b):
    result = 0
    z = 0
    x = np.zeros((n,1))
    pi = np.zeros((m,1))

    c_phase2 = np.concatenate((c, np.ones((m,1))))

    # phase 1 starts
    Binv = np.identity(m)
    phase1_c = np.zeros((n+m,1))
    phase1_c[n:] += 1

    varstatus = np.zeros((1,n)) - 1
    varstatus = varstatus.astype(int)
    xB = np.dot(Binv, b)
    cB = np.ones((m,1))
    basicvars = np.zeros((1, m))
    for i in range(m):
        basicvars[0][i] = n+i
    basicvars = basicvars.astype(int)

    for i in range(1000):
        cB_t = cB.T
        pi_t = np.dot(cB_t, Binv)

        # step 1
        s, ev, minrc = fullfindEV(n, phase1_c, A, varstatus, pi_t, phase1=True)

        if minrc >= 0:
            break

        a_s = A[:, ev].reshape(m,1)
        BinvAs = np.dot(Binv, a_s)

        # step 2
        r, lv, minratio = fullfindLV(m, xB, BinvAs, basicvars, phase1=True)

        # step 3
        varstatus, basicvars, cB, Binv, xB, _ = fullupdate(m, phase1_c, s, r, BinvAs, varstatus, basicvars, cB, Binv, xB, phase1=True)

        if (basicvars[0] < n).all():
            # print('break_condition reached')
            # using these print statements you can check the intermediate values in a iteration.
            # print(m, phase1_c, s, r, varstatus, basicvars, cB, Binv, xB)
            break


    # phase 1 ended; check if artificial in basis
    # print(f'basicvars:{basicvars}')
    # print(varstatus, basicvars)
    # print(Binv)

    while True:
        if (basicvars[0] < n).all():
            # print('break_condition reached')
            # print(m, c_phase2, s, r, varstatus, basicvars, cB, Binv, xB)
            break

        # step 1
        s, ev, minrc = fullfindEV(n, c_phase2, A, varstatus, pi_t, phase1=True)
        a_s = A[:, ev].reshape(m,1)
        BinvAs = np.dot(Binv, a_s)

        # step 2
        for index, value in enumerate(basicvars[0]):
            if value >= n:
                r, lv, minratio = index, value, 0
                break


        varstatus, basicvars, cB, Binv, xB, BinvAs = fullupdate(m, c_phase2, s, r, BinvAs, varstatus, basicvars, cB, Binv, xB, phase1=True)
        # print('after')
        # print(varstatus, basicvars)
        # print(xB)
        # print(Binv)
        # print(BinvAs)

    cB = phase1_c[basicvars[0]]
    z = np.sum(cB * xB)

    if z > 0:

        z = 0
        x = np.zeros_like(x)
        pi = np.zeros_like(pi_t.T)

        return [result, z, x, pi]

    # phase 2 starts
    result = -1
    cB = c_phase2[basicvars[0]]

    for i in range(1000):
        cB_t = cB.T
        pi_t = np.dot(cB_t, Binv)

        # step 1
        s, ev, minrc = fullfindEV(n, c_phase2, A, varstatus, pi_t, phase1=False)

        if minrc >= 0:
            result = +1
            break

        a_s = A[:, ev].reshape(m,1)
        BinvAs = np.dot(Binv, a_s)

        # step 2
        r, lv, minratio = fullfindLV(m, xB, BinvAs, basicvars, phase1=False)

        # step 3
        varstatus, basicvars, cB, Binv, xB, _ = fullupdate(m, c_phase2, s, r, BinvAs, varstatus, basicvars, cB, Binv, xB, phase1=False)

    pi = pi_t.T
    for i in range(pi.shape[0]):
      pi[i][0] = round(pi[i][0], 2)
    z = np.sum(cB * xB)

    for index, value in enumerate(varstatus[0]):
        value = int(value)
        if value == -1:
            continue
        else:
            x[index] = round(xB[value][0], 2)

    return [result, round(z, 2), x, pi]

In [6]:
m, n = 3, 7
c = np.array([1,10,3,4,0,0,0]).reshape(n,1)
A = np.array([
    [1,2,1,1,1,0,0],
    [-1,3,1,2,0,1,0],
    [-1,-1,0,-1,0,0,1]
])
b = np.array([4,6,1]).reshape(m,1)

In [7]:
[result, z, x, pi] = fullrsm(m,n,c,A,b)

In [8]:
print(f'result:{result}')
print(f'z:{z}')
print(f'solution:\n{x}')
print(f'pi:\n{pi}')

result:1
z:0.0
solution:
[[0.]
 [0.]
 [0.]
 [0.]
 [4.]
 [6.]
 [1.]]
pi:
[[0.]
 [0.]
 [0.]]
